We start by adding the necessary folders to the current working path.

In [1]:
import sys, os
import scipy.io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1]) # 2D folder

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")
# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")
# add FTLE folder to current working path
sys.path.append(parent_directory+"/demos/AdvectiveBarriers/FTLE2D")
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap
# Import function to compute finite time Lyapunov exponent (FTLE)
from ipynb.fs.defs.FTLE import _FTLE
# Import package for parallel computing
from joblib import Parallel, delayed

# Overview

In the following notebook we compute the fluctuation of the Finite Time Lyapunov Exponents ($ \mathrm{FTLE} $) on simulated velocity fields (tested with MITgcm results). The notebook is structured as follows:

1. Import data from the MITgcm results stored in the folder 'data/MITgcm'.
2. Define computational parameters and data.
3. Define spatio-temporal domain.
4. Interpolate velocity from (discrete) gridded data.
5. $ \mathrm{FTLE} $:

    Compute gradient of flow map $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $ over meshgrid using an auxiliary grid.
    
    Compute $ \mathrm{FTLE} $ from maximum singular value $ \sigma_{max} $ of $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $ according to:
    
    \begin{equation}
        \mathrm{FTLE}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{1}{t_N-t_0}\log(\sigma_{max}(\mathbf{x}_0)) \label{eq: FTLE} \tag{1}
    \end{equation}
       
6. References

# Import data

In [2]:
# Import velocity data from file in data-folder
mat_file = sio.loadmat('../../2D/data/MITgcm/Leman_2021-08-30_2021-09-11.mat')
date_ref = np.datetime64('2021-07-26T00:00:00')

U = mat_file['u'] # array (NY, NX, NT)
V = mat_file['v'] # array (NY, NX, NT)
x = mat_file['x'] # array (NY, NX)
y = mat_file['y'] # array (NY, NX)
time_data = mat_file['t'] # array (1, NT)

# Initial time (in seconds)
start_analysis = np.datetime64('2021-09-06T17:30:00')
t0 = (start_analysis - date_ref)/ np.timedelta64(1, 's') # float
#t0 = time_data[:,0][0] # float
# Final time (in seconds)
tN = time_data[:,-1][0] # float

# FTLE interval (in seconds)
ftle_interval = 24 * 3600
is_forward = False # =False if backward ftle

# Computing time step-size (in seconds)
dt = 3600 # float

if not is_forward:
    dt = -dt    
    
output_folder_name = f"{ftle_interval/3600}h_{'forward' if is_forward else 'backward'}_ftle_geneva"    

# Longitudinal and latitudinal boundaries (in degrees)
xmin = x.min() # float
xmax = x.max() # float
ymin = y.min() # float
ymax = y.max() # float

# Computational parameters and data

Here we define the computational parameters and the data.

In [3]:
# Number of cores for parallel computing
Ncores = 10 # int

# Time resolution of data
dt_data = time_data[0, 1]-time_data[0,0] # float

# Periodic boundary conditions
periodic_x = False # bool
periodic_y = False # bool
periodic_t = False # bool
periodic = [periodic_x, periodic_y, periodic_t]

# Unsteady velocity field
bool_unsteady = True # bool

# Defined domain
defined_domain = np.isfinite(U[:,:,0]).astype(int) # array (NY, NX)

## Compute meshgrid of dataset
X, Y = np.meshgrid(x, y) # array (NY, NX), array (NY, NX)

## Resolution of meshgrid
dx_data = X[0,1]-X[0,0] # float
dy_data = Y[1,0]-Y[0,0] # float

delta = [dx_data, dy_data] # list (2, )

# Spatio-temporal domain

Here we define the spatio-temporal domain over which to consider the dynamical system.

In [4]:
def check_boundaries_validity(X, Y, time_data, xmin, xmax, ymin, ymax, t0, tN):
    # make sure that domain is part of the data domain.
    assert np.min(X) <= xmin <= np.max(X), " xmin must be between "+f'{np.min(X)} and {np.max(X)}'
    assert np.min(X) <= xmax <= np.max(X), " xmax must be between "+f'{np.min(X)} and {np.max(X)}'
    assert np.min(Y) <= ymin <= np.max(Y), " ymin must be between "+f'{np.min(Y)} and {np.max(Y)}'
    assert np.min(Y) <= ymax <= np.max(Y), " ymax must be between "+f'{np.min(Y)} and {np.max(Y)}'
    assert np.min(time_data) <= t0 <= np.max(time_data), " t0 must be between "+f'{np.min(time_data)} and {np.max(time_data)}'
    assert np.min(time_data) <= tN <= np.max(time_data), " tN must be between "+f'{np.min(time_data)} and {np.max(time_data)}'

In [5]:
check_boundaries_validity(X, Y, time_data, xmin, xmax, ymin, ymax, t0, tN)

# Spacing of meshgrid (in degrees)
dx = dx_data # float
dy = dy_data # float

x_domain = np.arange(xmin, xmax + dx, dx) # array (Nx, )
y_domain = np.arange(ymin, ymax + dy, dy) # array (Ny, )

X_domain, Y_domain = np.meshgrid(x_domain, y_domain) # array (Ny, Nx)

Ny = X_domain.shape[0] # int
Nx = X_domain.shape[1] # int

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .1 # float between [1/100, 1/5]
aux_grid = [aux_grid_ratio*dx, aux_grid_ratio*dy] # list (2, )

# Interpolate velocity

In order to evaluate the velocity field at arbitrary locations and times, we interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". Default value is "cubic".

In [6]:
# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, method = "cubic")

Interpolant_u = Interpolant[0] # RectangularBivariateSpline-object
Interpolant_v = Interpolant[1] # RectangularBivariateSpline-object

# $ \mathrm{FTLE} $

Next, we compute the $ \mathrm{FTLE} $ over the meshgrid over the given time-interval.
We iterate over all initial conditions and first calculate the gradient of the flow map using an auxiliary grid. From the maximum singular value of the gradient of the flow map we can then compute the $ \mathrm{FTLE} $.

In [7]:
# Vectorize initial conditions by arranging them to a vector of size (Nx*Ny, )
x0 = X_domain.ravel() # array (Nx*Ny,)
y0 = Y_domain.ravel() # array (Nx*Ny,)

# Split x0, y0 into 'Ncores' equal batches for parallel computing
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

x0_batch = list(split(x0, Ncores)) # list (Nx*Ny)
y0_batch = list(split(y0, Ncores)) # list (Nx*Ny)

def parallel_FTLE(x0_batch, y0_batch, time):
    
    # Initial conditions
    X0 = np.array([x0_batch, y0_batch]) # array (2, Nx*Ny)

    # Compute gradient of flow map
    gradFmap = gradient_flowmap(time, X0, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid) # array (Nt, 2, 2, Nx*Ny)

    # Extract gradient from t0 to tN
    gradFmap_t0_tN = gradFmap[-1,:, :, :] # array (Nt, 2, 2, Nx*Ny)

    # Compute FTLE
    FTLE = [] # list (Nx*Ny,)
    for i in range(gradFmap_t0_tN.shape[2]):
        FTLE.append(_FTLE(gradFmap_t0_tN[:,:,i], abs(time[0]-time[-1])))
    
    return FTLE

In [8]:
def plot_and_save(fig_name, FTLE):    
    fig = plt.figure(figsize=(14, 6))
    ax = plt.axes()
    
    im = plt.imshow(FTLE, interpolation='nearest', vmin=0, vmax=1e-4)
    plt.gca().invert_yaxis()
    cbar = plt.colorbar(im)
    cbar.set_label('Finite Time Lyapunov Exponent (1/s)')
    plt.title(f'{fig_name}')
    
    # Axis Labels
    ax.set_xlabel("XC (m)")
    ax.set_ylabel("YC (m)")
    
    plt.tight_layout()
    plt.savefig(f'figures/{output_folder_name}/{fig_name}.png')
    plt.close(fig)

In [ ]:
for t_ini in np.arange(t0, tN - ftle_interval - abs(dt), abs(dt)):    
    if is_forward:
        t_start = t_ini
        t_end = t_ini+ftle_interval
    else:
        t_start = t_ini+ftle_interval
        t_end = t_ini
        
    time = np.arange(t_start, t_end+dt, dt) # shape (Nt,)
    
    # compute FTLE
    results = Parallel(n_jobs=Ncores, verbose = 50)(delayed(parallel_FTLE)(x0_batch[i], y0_batch[i], time) for i in range(len(x0_batch)))

    # Extract FTLE from results of parallel computing
    FTLE = results[0]
    
    for res in results[1:]:
        FTLE = np.append(FTLE, res)
        
    # Reshape array from vectorized form to structured meshgrid
    FTLE = FTLE.reshape((X_domain.shape[0], X_domain.shape[1])) # array (Ny, Nx)
    
    date_start =  pd.Timestamp(date_ref + np.timedelta64(int(t_start), 's')).strftime('%Y-%m-%d_%H-%M-%S')
    date_end =  pd.Timestamp(date_ref + np.timedelta64(int(t_end), 's')).strftime('%Y-%m-%d_%H-%M-%S')  
    
    fig_name = f"{'forward' if is_forward else 'backward'} FTLE {date_start} - {date_end}"
    pd.DataFrame(FTLE).to_csv(f'results/{output_folder_name}/{fig_name}.csv', index=False, header=False)
    plot_and_save(fig_name, FTLE)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.3min remaining:  5.0min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.1min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.5min remaining:  1.0min
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.6min remaining:   40.3s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.6min remaining:   24.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.8min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.8min
[Pa

[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min remaining:   57.6s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   37.3s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.5min remaining:   23.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.7min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.0min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min remaining:  1.9min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min remaining:   55.8s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   37.5s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.6min

[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.9min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.0min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min remaining:  2.0min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.5min remaining:   59.4s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.6min remaining:   40.5s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.6min remaining:   24.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.6min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  5.0min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.2min remaining:  2.9min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min

[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.6min remaining:   24.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.3min remaining:  5.1min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.1min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.5min remaining:   59.4s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   39.1s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.5min remaining:   22.8s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent wor

[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.4min remaining:  2.0min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.5min remaining:   58.8s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   39.5s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.5min remaining:   23.2s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.8min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  2.9min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min remaining:  2.0min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.8min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.3min remaining:  3.0min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min remaining:  2.0min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.4min remaining:  1.4min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min remaining:   56.8s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   38.7s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.6min remaining:   23.4s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.2min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.8min
[Pa

[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min remaining:   54.2s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   38.8s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.5min remaining:   22.8s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.7min finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:  1.2min remaining:  4.6min
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.2min remaining:  2.9min
[Parallel(n_jobs=10)]: Done   4 out of  10 | elapsed:  1.3min remaining:  1.9min
[Parallel(n_jobs=10)]: Done   5 out of  10 | elapsed:  1.3min remaining:  1.3min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed:  1.4min remaining:   55.6s
[Parallel(n_jobs=10)]: Done   7 out of  10 | elapsed:  1.5min remaining:   39.1s
[Parallel(n_jobs=10)]: Done   8 out of  10 | elapsed:  1.5min

In [ ]:
print('test')

# References

[1] Haller, G. (2001). Distinguished material surfaces and coherent structures in three-dimensional fluid flows. Physica D: Nonlinear Phenomena, 149(4), 248-277.

[2] Haller, G. (2015). Lagrangian coherent structures. Annual Review of Fluid Mechanics, 47, 137-162.

[3] Haller, G., & Sapsis, T. (2011). Lagrangian coherent structures and the smallest finite-time Lyapunov exponent. Chaos: An Interdisciplinary Journal of Nonlinear Science, 21(2), 023115.

[4] Notebook 5.1. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.